## Creating a Problem

In this notebook, we create and solve a single particle model (SPM). This is achieved using a predefined parameter set introduced in Marquis et al. [[1]](https://doi.org/10.1149/1945-7111/abbce4) 

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies.

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

import numpy as np

import pybop

pybop.PlotlyManager().pio.renderers.default = "notebook_connected"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Creating a Model

A fitting problem consists of three parts: An experimental dataset that is used for fitting, a model that the dataset should be fit to, and the parameters that should be fit. To obtain a problem we therefore first need to create a model. This has been explained in detail in the creating_a_model example. 

In [ ]:
parameter_set = pybop.ParameterSet.pybamm("Marquis2019")
model = pybop.lithium_ion.SPM(parameter_set=parameter_set)

Next, the parameters. All parameters of the problem are contained in pybop.Parameters(). Each of the variable to fit is defined as a pybop.Parameter() separated by commas inside pybop.Parameters(). In this example, we have two parameters, the positive electrode thickness and the positive particle radius:


In [ ]:
parameters = pybop.Parameters(
    pybop.Parameter(
        "Positive electrode thickness [m]",
        prior=pybop.Gaussian(7.56e-05, 0.05e-05),
        bounds=[65e-06, 10e-05],
    ),
    pybop.Parameter(
        "Positive particle radius [m]",
        prior=pybop.Gaussian(5.22e-06, 0.05e-06),
        bounds=[2e-06, 9e-06],
    ),
)

The third component is the dataset. This could be an experimental dataset or a simulation. In this example, we just use simulation data by running model.predict() for a 1C discharge, similar to the creating_a_problem example. To pass it to the problem, we need to extract the voltage, time and current data from the simulation results and define it as the dataset:

In [ ]:
t_eval = np.arange(0, 3700, 1)
solution = model.predict([], t_eval)  # No inputs i.e []
dataset = pybop.Dataset(
    {
        "Time [s]": np.arange(0, len(solution["Voltage [V]"].data), 1),
        "Current function [A]": solution["Current [A]"].data,
        "Voltage [V]": solution["Voltage [V]"].data,
    }
)

### Setting up the problem
With all parts setup, we can now make a problem. Therefore, we use pybop.FittingProblem() to define a simple fitting problem and the model, parameters and dataset defined before as inputs.

In [ ]:
problem = pybop.FittingProblem(model, parameters, dataset)

### Testing the problem
To verify our problem, we solve assign a sum of squared error cost function to it and solve it. 
The solution can be compared to the values in the parameter_set defined in the beginning.

In [ ]:
cost = pybop.SumSquaredError(problem)
optim = pybop.Optimisation(cost)
result = optim.run()
print(result)
print(parameter_set["Positive electrode thickness [m]"])
print(parameter_set["Positive particle radius [m]"])

(array([7.62324125e-05, 5.20703504e-06]), inf)
7.67500163945363e-05
5.310487559937693e-06
